In [1]:
import dnc
import torch
import torch.nn as nn
import time
import math
import numpy as np
import pickle

In [2]:
# dtype = torch.FloatTensor
dtype_float = torch.cuda.FloatTensor # Uncomment this to run on GPU
dtype_long = torch.cuda.LongTensor

# Model Definitions

In [3]:
class DNCEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, color_dim):
        super(DNCEncoder, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.embed = nn.Embedding(vocab_size, embed_dim).cuda()
        self.rnn = dnc.DNC(
          input_size=embed_dim + color_dim,
          hidden_size=128,
          rnn_type='lstm',
          num_layers=4,
          nr_cells=100,
          cell_size=32,
          read_heads=4,
          batch_first=True,
          gpu_id=0,
          debug=True
        )
        self.decoder = nn.Linear(embed_dim + color_dim, vocab_size).cuda() # don't predict over padding tag
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
        self.vocab_size = vocab_size
        
    def forward(self, caption, color):
        embeds = self.embed(caption).cuda()
        #embeds = nn.utils.pack_padded_sequence(embeds, caption_lens, batch_first=True)
        #print(embeds.type())
        #print(color.type())
        color_reps = color.repeat(1, caption.shape[1], 1) # repeat color for number of tokens in captions
        #print(color_reps)
        # concatenate colors to caption
        inputs = torch.cat((embeds, color_reps), dim=2) # cat along the innermost dimension
        #print(inputs.size())
        # dnc magic
        (controller_hidden, memory, read_vectors) = (None, None, None)
        output, (controller_hidden, memory, read_vectors), debug_memory = \
          rnn(inputs, (controller_hidden, memory, read_vectors), reset_experience=True)


        result = self.decoder(output).cuda()
        return self.logsoftmax(result)

In [4]:
class PytorchModel():
    def __init__(self,  model, optimizer=torch.optim.Adadelta,
                 criterion=nn.NLLLoss, lr=0.2, num_epochs=30, batch_size=1):
        """
        Right now this is kind of ugly because you have to pass literally all of the experiment arguments
        to this constructor. 
        """
        self.model = model

        self.optimizer = optimizer
        self.criterion = criterion
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # misc args:
        self.lr = lr
        self.num_epochs = num_epochs
        self.batch_size = batch_size

        # for reproducibility, store training pairs
        self.features  = None
        self.target = None

        # also make sure the model has been initialized before we do anything
        self.initialized = False

    def fit(self, X, y, validation_size = 3000):
        """
        The main sklearn-like function that takes input features X (in
        a list or array - we use color features and caption features
        for pytorch models) and targets, y, also a np array. The function
        just stores them and calls its internal train model function
        """

        # set up validation data if necessary
        if validation_size > 0:
            self.features = X[:-validation_size]
            self.targets = y[:-validation_size]
            self.val_features= X[-validation_size:]
            self.val_targets = y[-validation_size:]
        else:
            self.features = X
            self.targets = y 
            self.val_features = None
            self.val_features = None

        self.train_model()

    # from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def init_model(self, **model_params):
        """
        Interesting quirk - in most cases, the model takes the vocabulary size
        as an input, but there's no way for the user to know what the vocab
        size is before calling the init_model method. Even though it's kinda ugly,
        what we can do instead is pass all of the model params (named) minus
        the vocab size to this function, and it will create the model. I don't 
        really like this but it works for now
        """
        if self.initialized:
            return

        self.model = self.model(**model_params)
        self.initialized = True

    def train_iter(self, caption, colors, target, criterion):
        """
        Generate predictions based on the caption and color. Caculate loss against
        the target using the criterion and optimize using the optimizer. All subclasses
        will do this a little bit differently, so do implement this in the subclasses
        """
        pass


    def train_model(self):
        if not self.initialized:
            print("Make sure you initialize the model with the parameters you want")
            return


        optimizer = self.optimizer(lr=self.lr, params=self.model.parameters())
        criterion = self.criterion()

        start_time = time.time()
        store_losses_every = 100
        print_losses_every = 10
        self.stored_losses = [] # theoretically we store losses so we can plot them later - 
                                # I don't think this part of the code works though. What we
                                # can do instead is take a few thousand or so training examples
                                # out and use them for "evaluation" every 1 epoch or so
        for epoch in range(self.num_epochs):
            print("---EPOCH {}---".format(epoch))
            stored_loss_total = 0
            print_loss_total = 0

            for i, pair in enumerate(self.features):
                caption, colors = pair
                if self.batch_size == 1:
                    caption = torch.tensor([caption]).type(dtype_long)
                    colors = torch.tensor([colors]).type(dtype_float)
                    target = torch.tensor([self.targets[i]]) 
                else:
                    caption = torch.tensor(caption).type(dtype_long)
                    colors = torch.tensor(colors).type(dtype_float)
                    target = torch.tensor(self.targets[i]).type(dtype_long)


                optimizer.zero_grad()
                loss = self.train_iter(caption, colors, target, criterion).cuda()
                loss.backward()
                optimizer.step()
                stored_loss_total += loss.item()
                print_loss_total += loss.item()

                if i % print_losses_every == 0:
                    print_loss_avg = print_loss_total / print_losses_every
                    print("{} ({}:{} {:.2f}%) {:.4f}".format(self.asMinutes(time.time() - start_time),
                                                      epoch, i, i/len(self.features)*100,
                                                      print_loss_avg))
                    print_loss_total = 0

                if i % store_losses_every == 0:
                    stored_loss_avg = stored_loss_total / store_losses_every
                    self.stored_losses.append(stored_loss_avg)
                    stored_loss_total = 0

            # run validation data after every epoch
            self.save_model("checkpoint_{}.params".format(epoch))
            self.validate_model(epoch, criterion)

    def validate_model(self, epoch_num, criterion):
        if self.val_features is None:
            return

        # run validation
        total_loss = 0
        self.model.eval()
        with torch.no_grad():
            for i, pair in enumerate(self.val_features):
                caption, colors = pair
                if self.batch_size == 1:
                    caption = torch.tensor([caption]).type(dtype_long)
                    colors = torch.tensor([colors]).type(dtype_float)
                    target = torch.tensor([self.targets[i]]) 
                else:
                    caption = torch.tensor(caption).type(dtype_long)
                    colors = torch.tensor(colors).type(dtype_float)
                    target = torch.tensor(self.targets[i]) 
                 
                total_loss += self.train_iter(caption, colors, target, criterion).item()
            print("="*25)
            print("AFTER EPOCH {} - AVERAGE VALIDATION LOSS: {}".format(i, total_loss / len(self.val_features)))
            print("="*25)
        self.model.train()


    def load_model(self, filename):
        """
        Load model from saved file at filename
        """
        if not self.initialized:
            self.init_model
        self.model.load_state_dict(torch.load(filename))

    def save_model(self, filename):
        """
        Save model to file at filename
        """
        torch.save(self.model.state_dict(), filename)

In [5]:
# shamelessly copied from beam search code
from queue import PriorityQueue

class BeamNode():
    def __init__(self, log_prob, tokens, ended):
        self.log_prob = log_prob
        self.tokens = tokens
        self.ended = ended

    def score(self):
        return -self.log_prob

    def __eq__(self, other):
        return self.score() == other.score()

    def __lt__(self, other):
        return self.score() < other.score()
    

class DNCSpeaker(PytorchModel):
    def predict(self, X, sample=1, beam_width=5):
        all_tokens = []
        self.model.eval()
        max_gen_len = 20
    
        self.model.eval()
        with torch.no_grad():
            for i, feature in enumerate(X):
                caption, colors = feature
                if self.batch_size == 1:
                    caption = torch.tensor([caption]).type(dtype_long)
                    colors = torch.tensor([colors]).type(dtype_float)
                else:
                    caption = torch.tensor([caption]).type(dtype_long)
                    colors = torch.tensor([colors]).type(dtype_float)

    
                beam_nodes = PriorityQueue()
                ended_list = []
    
                tokens = caption[:, 0].view(-1, 1) # begin at start token
                start = BeamNode(0, tokens, False)
                beam_nodes.put(start)
    
                for i in range(max_gen_len + 1):
                    node = beam_nodes.get()
                    if node.ended:
                        ended_list.append(np.array(node.tokens[0].numpy()))
                        if len(ended_list) == sample:
                            break
                    else:
                        tokens = node.tokens
                        vocab_preds = self.model(colors, tokens)[:,-1:,:] # just distribution over last token
                        log_probs, prediction_indices = vocab_preds.topk(beam_width, dim=2)  # taking the topk predictions
                        for j in range(beam_width):
                            prediction_index = prediction_indices[:,-1,j:j+1] # a single prediction
                            log_prob = log_probs[0][0][j].item()
                            updated_tokens = tokens.clone()
                            updated_tokens = torch.cat((updated_tokens, prediction_index), dim=1)
                            updated_log_prob = node.log_prob + log_prob
                            ended = ((i == max_gen_len - 1) or (prediction_index.item() == caption[:, -1].view(-1, 1)))
                            new_node = BeamNode(updated_log_prob, updated_tokens, ended)
                            beam_nodes.put(new_node)
                if sample == 1: # for backwards compatability
                    all_tokens.append(np.array(ended_list))
                else:
                    all_tokens.append(ended_list)
        return all_tokens
    
    def train_iter(self, caption_tensor, color_tensor, target_tensor, criterion):
        model_output = self.model(caption_tensor, color_tensor).cuda()
        
        #model_output = model_output[:, :-1, :].squeeze(0)
        model_output = model_output.view(-1, self.model.vocab_size)
        target_tensor = target_tensor.view(-1)
        #print("output shape:", model_output.size())
        #print("target shape:", target_tensor.size())
        loss = criterion(model_output, target_tensor).cuda()
        return loss

In [6]:
# load data
train_features_1 = None
with open("train_features_1.pkl", "rb") as file:
    train_features_1 = pickle.load(file)
    
train_features_2 = None
with open("train_features_2.pkl", "rb") as file:
    train_features_2 = pickle.load(file)
    
train_features = train_features_1 + train_features_2

train_targets = None
with open("train_targets.pkl", "rb") as file:
    train_targets = pickle.load(file)
    
idx2word = None
with open("idx2word.pkl", "rb") as file:
    idx2word = pickle.load(file)

In [7]:
# define rnn
rnn = dnc.DNC(
  input_size=103,#64,
  hidden_size=128,
  rnn_type='lstm',
  num_layers=4,
  nr_cells=100,
  cell_size=32,
  read_heads=4,
  batch_first=True,
  gpu_id=0,
  debug=True
)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# load model
dnc_speaker = DNCSpeaker(DNCEncoder, num_epochs=5, batch_size=128)
dnc_speaker.init_model(vocab_size = len(idx2word), embed_dim=100, color_dim=3)
dnc_speaker.load_model("checkpoint_1.params")

In [10]:
# train model
dnc_speaker.fit(train_features, train_targets, validation_size=0) # things break if validation size is nonzero for some reason

---EPOCH 0---
0m 0s (0:0 0.00%) 0.1366
0m 1s (0:10 0.08%) 1.3653
0m 3s (0:20 0.17%) 1.3650
0m 5s (0:30 0.25%) 1.3646
0m 6s (0:40 0.34%) 1.3638
0m 8s (0:50 0.42%) 1.3627
0m 10s (0:60 0.50%) 1.3625
0m 12s (0:70 0.59%) 1.3616
0m 13s (0:80 0.67%) 1.3609
0m 15s (0:90 0.76%) 1.3605
0m 17s (0:100 0.84%) 1.3602
0m 18s (0:110 0.92%) 1.3589
0m 20s (0:120 1.01%) 1.3573
0m 22s (0:130 1.09%) 1.3574
0m 23s (0:140 1.18%) 1.3567
0m 25s (0:150 1.26%) 1.3561
0m 27s (0:160 1.34%) 1.3554
0m 29s (0:170 1.43%) 1.3558
0m 30s (0:180 1.51%) 1.3537
0m 32s (0:190 1.60%) 1.3540
0m 34s (0:200 1.68%) 1.3530
0m 36s (0:210 1.76%) 1.3525
0m 37s (0:220 1.85%) 1.3516
0m 39s (0:230 1.93%) 1.3513
0m 41s (0:240 2.02%) 1.3503
0m 43s (0:250 2.10%) 1.3495
0m 44s (0:260 2.19%) 1.3490
0m 46s (0:270 2.27%) 1.3477
0m 48s (0:280 2.35%) 1.3476
0m 50s (0:290 2.44%) 1.3472
0m 52s (0:300 2.52%) 1.3463
0m 54s (0:310 2.61%) 1.3457
0m 55s (0:320 2.69%) 1.3452
0m 57s (0:330 2.77%) 1.3440
0m 59s (0:340 2.86%) 1.3439
1m 0s (0:350 2.94%) 1.3

8m 2s (0:2820 23.70%) 1.2385
8m 4s (0:2830 23.78%) 1.2374
8m 5s (0:2840 23.87%) 1.2364
8m 7s (0:2850 23.95%) 1.2362
8m 9s (0:2860 24.04%) 1.2356
8m 10s (0:2870 24.12%) 1.2352
8m 12s (0:2880 24.20%) 1.2348
8m 14s (0:2890 24.29%) 1.2346
8m 16s (0:2900 24.37%) 1.2341
8m 17s (0:2910 24.46%) 1.2341
8m 19s (0:2920 24.54%) 1.2335
8m 21s (0:2930 24.62%) 1.2339
8m 22s (0:2940 24.71%) 1.2319
8m 24s (0:2950 24.79%) 1.2336
8m 26s (0:2960 24.88%) 1.2335
8m 27s (0:2970 24.96%) 1.2343
8m 29s (0:2980 25.04%) 1.2321
8m 31s (0:2990 25.13%) 1.2316
8m 33s (0:3000 25.21%) 1.2340
8m 35s (0:3010 25.30%) 1.2327
8m 37s (0:3020 25.38%) 1.2318
8m 39s (0:3030 25.46%) 1.2309
8m 40s (0:3040 25.55%) 1.2294
8m 42s (0:3050 25.63%) 1.2295
8m 44s (0:3060 25.72%) 1.2308
8m 45s (0:3070 25.80%) 1.2294
8m 47s (0:3080 25.88%) 1.2297
8m 49s (0:3090 25.97%) 1.2293
8m 51s (0:3100 26.05%) 1.2295
8m 53s (0:3110 26.14%) 1.2293
8m 54s (0:3120 26.22%) 1.2278
8m 56s (0:3130 26.30%) 1.2277
8m 58s (0:3140 26.39%) 1.2282
8m 59s (0:3150 

15m 41s (0:5510 46.31%) 1.1821
15m 43s (0:5520 46.39%) 1.1811
15m 45s (0:5530 46.47%) 1.1822
15m 47s (0:5540 46.56%) 1.1803
15m 48s (0:5550 46.64%) 1.1803
15m 50s (0:5560 46.73%) 1.1791
15m 52s (0:5570 46.81%) 1.1796
15m 53s (0:5580 46.89%) 1.1801
15m 55s (0:5590 46.98%) 1.1802
15m 57s (0:5600 47.06%) 1.1803
15m 58s (0:5610 47.15%) 1.1791
16m 0s (0:5620 47.23%) 1.1771
16m 2s (0:5630 47.31%) 1.1799
16m 3s (0:5640 47.40%) 1.1796
16m 5s (0:5650 47.48%) 1.1795
16m 7s (0:5660 47.57%) 1.1794
16m 8s (0:5670 47.65%) 1.1800
16m 10s (0:5680 47.74%) 1.1783
16m 12s (0:5690 47.82%) 1.1788
16m 13s (0:5700 47.90%) 1.1784
16m 15s (0:5710 47.99%) 1.1794
16m 17s (0:5720 48.07%) 1.1769
16m 19s (0:5730 48.16%) 1.1773
16m 20s (0:5740 48.24%) 1.1784
16m 22s (0:5750 48.32%) 1.1766
16m 24s (0:5760 48.41%) 1.1762
16m 25s (0:5770 48.49%) 1.1779
16m 27s (0:5780 48.58%) 1.1776
16m 29s (0:5790 48.66%) 1.1766
16m 30s (0:5800 48.74%) 1.1769
16m 32s (0:5810 48.83%) 1.1765
16m 34s (0:5820 48.91%) 1.1760
16m 35s (0:583

23m 13s (0:8170 68.66%) 1.1482
23m 15s (0:8180 68.75%) 1.1459
23m 16s (0:8190 68.83%) 1.1486
23m 18s (0:8200 68.91%) 1.1467
23m 20s (0:8210 69.00%) 1.1477
23m 22s (0:8220 69.08%) 1.1490
23m 23s (0:8230 69.17%) 1.1475
23m 25s (0:8240 69.25%) 1.1486
23m 27s (0:8250 69.33%) 1.1475
23m 28s (0:8260 69.42%) 1.1473
23m 30s (0:8270 69.50%) 1.1483
23m 32s (0:8280 69.59%) 1.1483
23m 33s (0:8290 69.67%) 1.1474
23m 35s (0:8300 69.75%) 1.1470
23m 37s (0:8310 69.84%) 1.1456
23m 39s (0:8320 69.92%) 1.1473
23m 40s (0:8330 70.01%) 1.1446
23m 42s (0:8340 70.09%) 1.1458
23m 44s (0:8350 70.17%) 1.1477
23m 45s (0:8360 70.26%) 1.1472
23m 47s (0:8370 70.34%) 1.1475
23m 49s (0:8380 70.43%) 1.1468
23m 50s (0:8390 70.51%) 1.1462
23m 52s (0:8400 70.59%) 1.1447
23m 54s (0:8410 70.68%) 1.1473
23m 56s (0:8420 70.76%) 1.1457
23m 57s (0:8430 70.85%) 1.1456
23m 59s (0:8440 70.93%) 1.1462
24m 1s (0:8450 71.01%) 1.1464
24m 2s (0:8460 71.10%) 1.1445
24m 4s (0:8470 71.18%) 1.1460
24m 6s (0:8480 71.27%) 1.1440
24m 7s (0:84

30m 46s (0:10810 90.85%) 1.1266
30m 48s (0:10820 90.93%) 1.1260
30m 50s (0:10830 91.02%) 1.1255
30m 52s (0:10840 91.10%) 1.1242
30m 54s (0:10850 91.18%) 1.1253
30m 56s (0:10860 91.27%) 1.1262
30m 57s (0:10870 91.35%) 1.1285
30m 59s (0:10880 91.44%) 1.1249
31m 1s (0:10890 91.52%) 1.1265
31m 2s (0:10900 91.60%) 1.1261
31m 4s (0:10910 91.69%) 1.1266
31m 6s (0:10920 91.77%) 1.1282
31m 8s (0:10930 91.86%) 1.1261
31m 9s (0:10940 91.94%) 1.1253
31m 11s (0:10950 92.02%) 1.1247
31m 13s (0:10960 92.11%) 1.1241
31m 14s (0:10970 92.19%) 1.1241
31m 16s (0:10980 92.28%) 1.1250
31m 18s (0:10990 92.36%) 1.1215
31m 19s (0:11000 92.44%) 1.1233
31m 21s (0:11010 92.53%) 1.1258
31m 23s (0:11020 92.61%) 1.1276
31m 24s (0:11030 92.70%) 1.1232
31m 26s (0:11040 92.78%) 1.1239
31m 28s (0:11050 92.86%) 1.1211
31m 30s (0:11060 92.95%) 1.1235
31m 31s (0:11070 93.03%) 1.1246
31m 33s (0:11080 93.12%) 1.1236
31m 35s (0:11090 93.20%) 1.1263
31m 36s (0:11100 93.29%) 1.1256
31m 38s (0:11110 93.37%) 1.1257
31m 40s (0:111

...

135m 51s (4:60 0.50%) 1.0208
135m 53s (4:70 0.59%) 1.0254
135m 55s (4:80 0.67%) 1.0230
135m 57s (4:90 0.76%) 1.0263
135m 58s (4:100 0.84%) 1.0219
136m 0s (4:110 0.92%) 1.0179
136m 2s (4:120 1.01%) 1.0185
136m 3s (4:130 1.09%) 1.0202
136m 5s (4:140 1.18%) 1.0228
136m 7s (4:150 1.26%) 1.0195
136m 8s (4:160 1.34%) 1.0211
136m 10s (4:170 1.43%) 1.0260
136m 12s (4:180 1.51%) 1.0179
136m 13s (4:190 1.60%) 1.0255
136m 15s (4:200 1.68%) 1.0240
136m 17s (4:210 1.76%) 1.0189
136m 18s (4:220 1.85%) 1.0214
136m 20s (4:230 1.93%) 1.0219
136m 22s (4:240 2.02%) 1.0189
136m 23s (4:250 2.10%) 1.0195
136m 25s (4:260 2.19%) 1.0224
136m 27s (4:270 2.27%) 1.0212
136m 28s (4:280 2.35%) 1.0200
136m 30s (4:290 2.44%) 1.0245
136m 32s (4:300 2.52%) 1.0219
136m 34s (4:310 2.61%) 1.0211
136m 36s (4:320 2.69%) 1.0220
136m 38s (4:330 2.77%) 1.0187
136m 39s (4:340 2.86%) 1.0242
136m 41s (4:350 2.94%) 1.0175
136m 43s (4:360 3.03%) 1.0224
136m 44s (4:370 3.11%) 1.0247
136m 46s (4:380 3.19%) 1.0185
136m 48s (4:390 3.28

143m 19s (4:2710 22.78%) 1.0185
143m 21s (4:2720 22.86%) 1.0243
143m 22s (4:2730 22.94%) 1.0201
143m 24s (4:2740 23.03%) 1.0201
143m 26s (4:2750 23.11%) 1.0156
143m 27s (4:2760 23.20%) 1.0232
143m 29s (4:2770 23.28%) 1.0145
143m 31s (4:2780 23.36%) 1.0204
143m 32s (4:2790 23.45%) 1.0215
143m 34s (4:2800 23.53%) 1.0161
143m 36s (4:2810 23.62%) 1.0228
143m 38s (4:2820 23.70%) 1.0250
143m 39s (4:2830 23.78%) 1.0196
143m 41s (4:2840 23.87%) 1.0201
143m 43s (4:2850 23.95%) 1.0154
143m 44s (4:2860 24.04%) 1.0187
143m 46s (4:2870 24.12%) 1.0181
143m 48s (4:2880 24.20%) 1.0152
143m 50s (4:2890 24.29%) 1.0188
143m 52s (4:2900 24.37%) 1.0183
143m 53s (4:2910 24.46%) 1.0191
143m 55s (4:2920 24.54%) 1.0194
143m 57s (4:2930 24.62%) 1.0191
143m 58s (4:2940 24.71%) 1.0160
144m 0s (4:2950 24.79%) 1.0192
144m 1s (4:2960 24.88%) 1.0184
144m 3s (4:2970 24.96%) 1.0255
144m 5s (4:2980 25.04%) 1.0175
144m 7s (4:2990 25.13%) 1.0159
144m 8s (4:3000 25.21%) 1.0254
144m 10s (4:3010 25.30%) 1.0228
144m 12s (4:30

150m 39s (4:5290 44.46%) 1.0194
150m 40s (4:5300 44.54%) 1.0119
150m 42s (4:5310 44.63%) 1.0196
150m 44s (4:5320 44.71%) 1.0118
150m 45s (4:5330 44.79%) 1.0152
150m 47s (4:5340 44.88%) 1.0154
150m 49s (4:5350 44.96%) 1.0170
150m 50s (4:5360 45.05%) 1.0166
150m 52s (4:5370 45.13%) 1.0146
150m 54s (4:5380 45.21%) 1.0170
150m 55s (4:5390 45.30%) 1.0121
150m 57s (4:5400 45.38%) 1.0175
150m 59s (4:5410 45.47%) 1.0151
151m 0s (4:5420 45.55%) 1.0154
151m 2s (4:5430 45.63%) 1.0149
151m 4s (4:5440 45.72%) 1.0153
151m 5s (4:5450 45.80%) 1.0135
151m 7s (4:5460 45.89%) 1.0158
151m 9s (4:5470 45.97%) 1.0160
151m 10s (4:5480 46.05%) 1.0157
151m 12s (4:5490 46.14%) 1.0151
151m 14s (4:5500 46.22%) 1.0127
151m 15s (4:5510 46.31%) 1.0146
151m 17s (4:5520 46.39%) 1.0166
151m 19s (4:5530 46.47%) 1.0195
151m 20s (4:5540 46.56%) 1.0165
151m 22s (4:5550 46.64%) 1.0149
151m 24s (4:5560 46.73%) 1.0129
151m 25s (4:5570 46.81%) 1.0151
151m 27s (4:5580 46.89%) 1.0151
151m 29s (4:5590 46.98%) 1.0151
151m 30s (4:56

157m 56s (4:7870 66.14%) 1.0125
157m 58s (4:7880 66.22%) 1.0097
157m 59s (4:7890 66.31%) 1.0105
158m 1s (4:7900 66.39%) 1.0144
158m 3s (4:7910 66.48%) 1.0122
158m 4s (4:7920 66.56%) 1.0113
158m 6s (4:7930 66.64%) 1.0074
158m 8s (4:7940 66.73%) 1.0118
158m 9s (4:7950 66.81%) 1.0138
158m 11s (4:7960 66.90%) 1.0162
158m 13s (4:7970 66.98%) 1.0137
158m 15s (4:7980 67.06%) 1.0134
158m 17s (4:7990 67.15%) 1.0101
158m 18s (4:8000 67.23%) 1.0119
158m 20s (4:8010 67.32%) 1.0154
158m 22s (4:8020 67.40%) 1.0128
158m 23s (4:8030 67.48%) 1.0144
158m 25s (4:8040 67.57%) 1.0134
158m 27s (4:8050 67.65%) 1.0053
158m 29s (4:8060 67.74%) 1.0104
158m 30s (4:8070 67.82%) 1.0127
158m 32s (4:8080 67.90%) 1.0084
158m 34s (4:8090 67.99%) 1.0158
158m 35s (4:8100 68.07%) 1.0126
158m 37s (4:8110 68.16%) 1.0146
158m 39s (4:8120 68.24%) 1.0117
158m 40s (4:8130 68.33%) 1.0062
158m 42s (4:8140 68.41%) 1.0164
158m 44s (4:8150 68.49%) 1.0104
158m 45s (4:8160 68.58%) 1.0088
158m 47s (4:8170 68.66%) 1.0107
158m 49s (4:81

165m 13s (4:10440 87.74%) 1.0074
165m 15s (4:10450 87.82%) 1.0072
165m 17s (4:10460 87.91%) 1.0065
165m 18s (4:10470 87.99%) 1.0068
165m 20s (4:10480 88.07%) 1.0099
165m 22s (4:10490 88.16%) 1.0087
165m 23s (4:10500 88.24%) 1.0107
165m 25s (4:10510 88.33%) 1.0098
165m 27s (4:10520 88.41%) 1.0090
165m 28s (4:10530 88.49%) 1.0124
165m 30s (4:10540 88.58%) 1.0103
165m 32s (4:10550 88.66%) 1.0080
165m 34s (4:10560 88.75%) 1.0090
165m 35s (4:10570 88.83%) 1.0088
165m 37s (4:10580 88.92%) 1.0084
165m 39s (4:10590 89.00%) 1.0085
165m 40s (4:10600 89.08%) 1.0057
165m 42s (4:10610 89.17%) 1.0072
165m 44s (4:10620 89.25%) 1.0090
165m 45s (4:10630 89.34%) 1.0072
165m 47s (4:10640 89.42%) 1.0085
165m 49s (4:10650 89.50%) 1.0123
165m 51s (4:10660 89.59%) 1.0105
165m 53s (4:10670 89.67%) 1.0087
165m 55s (4:10680 89.76%) 1.0054
165m 56s (4:10690 89.84%) 1.0104
165m 58s (4:10700 89.92%) 1.0043
166m 0s (4:10710 90.01%) 1.0088
166m 1s (4:10720 90.09%) 1.0051
166m 3s (4:10730 90.18%) 1.0092
166m 5s (4:10

In [12]:
dnc_speaker.validate_model(1, dnc_speaker.criterion())